In [10]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from xgboost.sklearn import XGBClassifier
from pyod.models.knn import KNN

In [30]:
def get_precision(y, y_pred):
    
    outlier_percent = np.count_nonzero(y) / len(y)
    threshold = np.percentile(y_pred, 100 * (1 - outlier_percent))
    y_pred = (y_pred > threshold).astype("int")
    return precision_score(y, y_pred)

def random_selection(X, X_new, roc_list, p):
    import random
    selected_feature_rnd = random.sample(range(0, len(roc_list)), p)
    X_new_rnd = X_new[:, selected_feature_rnd]
    X_all_rnd = np.concatenate([X, X_new_rnd], axis = 1)
    return X_new_rnd, X_all_rnd

def accurate_selection(X, X_new, roc_list, p, n):
    roc_list = np.asarray(roc_list)
    length = roc_list.shape[0]
    roc_sorted = np.partition(roc_list, length - n)
    threshold = roc_sorted[int(length - n)]
    selected_feature_accurate = np.where(np.greater_equal(roc_list, threshold))
    X_new_accurate = X_new[:, selected_feature_accurate[0][0:p]]
    X_all_accurate = np.concatenate((X, X_new_accurate), axis = 1)
    return X_new_accurate, X_all_accurate

def balance_selection(X, X_new, roc_list, p):
    from scipy.stats import pearsonr
    
    selected_feature_balance = []
    pearson_list = np.zeros([len(roc_list), 1])
    max_value_idx = roc_list.index(max(roc_list))
    selected_feature_balance.append(max_value_idx)
    roc_list[max_value_idx] = -1
    
    for i in range(p - 1):
        for j in range(len(roc_list)):
            pearson = pearsonr(X_new[:, max_value_idx], X_new[:, j])
            pearson_list[j] = np.abs([pearson_list[j]]) + np.abs(pearson[0])
        discounted_roc = np.true_divide(roc_list, pearson_list.transpose())
        max_value_idx = np.argmax(discounted_roc)
        selected_feature_balance.append(max_value_idx)
        roc_list[max_value_idx] = -1
    
    X_new_balance = X_new[:, selected_feature_balance]
    X_all_balance = np.concatenate((X, X_new_balance), axis = 1)
    return X_new_balance, X_all_balance

def set_parameters(origin_label, iteration = 10, test_size = 0.3):
    
    # Parameter setting
    iteration = iteration
    test_size = test_size
    
    # Outlier percentage
    outlier_percentage = np.count_nonzero(origin_label) / len(origin_label)
    
    return iteration, test_size, outlier_percentage

def initialize_classifier(
    clf_list = [XGBClassifier(), RandomForestClassifier()],
    clf_name_list = ["Extreme_Grandient_Boost_", "Random_Forest_"],
    feature_name_list = [
        "origin", 
        "anomaly_score", "anomaly_score_random", "anomaly_score_accurate", "anomaly_score_balance",
        "semi_supervised", "semi_supervised_random", "semi_supervised_accurate", "semi_supervised_balance"
    ]
):
    result_dict = {}
    evaluation_list = ["roc_", "precision_", "auc_"]
    for clf_name in clf_name_list:
        for evaluation in evaluation_list:
            for feature in feature_name_list:
                result_dict[clf_name + evaluation + feature] = []
    
    return clf_list, result_dict

def get_results_from_KNN(k_list_def, origin_feature, origin_feature_test, label_test, outlier_percentag, method):
    
    k_list = [k for k in k_list_def if k < X.shape[0]]    
    train_array = np.zeros([origin_feature.shape[0], len(k_list)])
    test_array = np.zeros([origin_feature_test.shape[0], len(k_list)])
    feature_list = []
    roc_list = []
    precision_list = []
        
    for i in range(len(k_list)):
        k = k_list[i]
        clf = KNN(n_neighbors = k, contamination = outlier_percent, method = method)
        clf.fit(origin_feature)
        train_score = clf.decision_scores_
        predict_score = clf.decision_function(origin_feature_test)
        
        roc = np.round(roc_auc_score(label_test, predict_score), decimals = 4)
        precision = np.round(get_precision(label_test, predict_score), decimals = 4)
        print(
            "{method} KNN Roc and Precision in {k} Neighbors are {roc} and {precision}".format(
                method = method, k = k, roc = roc, precision = precision
            )
        )
        
        feature_list.append("KNN_" + method + "_" + str(k))
        roc_list.append(roc)
        precision_list.append(precision)
        train_array[:, i] = train_score
        test_array[:, i] = predict_score.ravel()
            
    return feature_list, train_array, test_array, roc_list, precision_list

def get_results_from_LOF(k_list_def, origin_feature, origin_feature_test, label_test, outlier_percentage):
    
    k_list = [k for k in k_list_def if k < X.shape[0]]
    train_array = np.zeros([origin_feature.shape[0], len(k_list)])
    test_array = np.zeros([origin_feature_test.shape[0], len(k_list)])
    feature_list = []
    roc_list = []
    precision_list = []
    
    for i in range(len(k_list)):
        k = k_list[i]
        clf = LocalOutlierFactor(n_neighbors = k)
        clf.fit(origin_feature)
        train_score = clf.negative_outlier_factor_ * -1
        predict_score = clf._decision_function(origin_feature_test)
        
        roc = np.round(roc_auc_score(label_test, predict_score), decimals = 4)
        precision = np.round(get_precision(label_test, predict_score), decimals = 4)
        print(
            "Local Outlier Factor Roc and Precision in {k} Neighbors are {roc} and {precision}".format(
                k = k, roc = roc, precision = precision
            )
        )

        feature_list.append("LOF_" + str(k))
        roc_list.append(roc)
        precision_list.append(precision)
        train_array[:, i] = train_score
        test_array[:, i] = predict_score.ravel()
    
    return feature_list, train_array, test_array, roc_list, precision_list

def get_results_from_SVM(nu_list, origin_feature, origin_feature_test, label_test, outlier_percentage):
        
    train_array = np.zeros([origin_feature.shape[0], len(nu_list)])
    test_array = np.zeros([origin_feature_test.shape[0], len(nu_list)])
    feature_list= []
    roc_list = []
    precision_list = []
    
    for i in range(len(nu_list)):
        nu = nu_list[i]
        clf = OneClassSVM(nu = nu)
        clf.fit(origin_feature)
        train_score = clf.decision_function(X) * -1
        predict_score = clf.decision_function(origin_feature_test)
        
        roc = np.round(roc_auc_score(label_test, predict_score), decimals = 4)
        precision = np.round(get_precision(label_test, predict_score), decimals = 4)
        print(
            "Support Vector Machine Roc and Precision in {nu} are {roc} and {precision}".format(
                nu = nu, roc = roc, precision = precision
            )
        )
        feature_list.append("SVM_" + str(nu))
        roc_list.append(roc)
        precision_list.append(precision)
        
        train_array[:, i] = train_score
        test_array[:, i] = predict_score.ravel()
    
    return feature_list, train_array, test_array, roc_list, precision_list

def get_results_from_IF(n_list, origin_feature, orgin_feature_test, label_test, outlier_percentage):
    
    train_array = np.zeros([origin_feature.shape[0], len(n_list)])
    test_array = np.zeros([orgin_feature_test.shape[0], len(n_list)])
    feature_list = []
    roc_list = []
    precision_list = []
    
    for i in range(len(n_list)):
        n = n_list[i]
        clf = IsolationForest(n_estimators = n)
        clf.fit(origin_feature)
        train_score = clf.decision_function(X) * -1
        predict_score = clf.decision_function(orgin_feature_test)
        
        roc = np.round(roc_auc_score(label_test, predict_score), decimals = 4)
        precision = np.round(get_precision(label_test, predict_score), decimals = 4)
        print(
            "Isolation Forest Roc and Precision in {n} Estimators are {roc} and {precision}".format(
                n = n, roc = roc, precision = precision
            )
        )
        
        feature_list.append("IF_" + str(n))
        roc_list.append(roc)
        precision_list.append(precision)
        
        train_array[:, i] = train_score
        test_array[:, i] = predict_score.ravel()
    
    return feature_list, train_array, test_array, roc_list, precision_list

def generate_new_feature(origin_feature, origin_feature_test, anomaly_score, anomaly_score_test, roc, precision):
    
    new_feature = np.concatenate([score for score in anomaly_score], axis = 1)
    new_feature_test = np.concatenate([score_test for score_test in anomaly_score_test], axis = 1)
    all_feature = np.concatenate([origin_feature, new_feature], axis = 1)
    all_feature_test = np.concatenate([origin_feature_test, new_feature_test], axis = 1)

    roc_list = list(np.concatenate([r for r in roc], axis = 0))
    precision_list = list(np.concatenate([prec for prec in precision], axis = 0))
    
    return new_feature, new_feature_test, all_feature, all_feature_test, roc_list, precision_list


def train_classifier(
    feature_list, feature_test_list, label, label_test,
    clf_list = [XGBClassifier(), RandomForestClassifier()],
    clf_name_list = ["Extreme_Grandient_Boost_", "Random_Forest_"],
    feature_name_list = [
        "origin", 
        "anomaly_score", "anomaly_score_random", "anomaly_score_accurate", "anomaly_score_balance",
        "semi_supervised", "semi_supervised_random", "semi_supervised_accurate", "semi_supervised_balance"
    ]
):
    
    for clf, clf_name in zip(clf_list, clf_name_list):
        print("Processing", clf_name)
        for i, feature_name in enumerate(feature_name_list):
            clf.fit(feature_list[i], label.ravel())
            y_predict = clf.predict_proba(feature_test_list[i])
            roc_score = roc_auc_score(label_test, y_predict[:, 1])
            precis = get_precision(label_test, y_predict[:, 1])
            precision, recall, _ = precision_recall_curve(label_test, y_predict[:, 1])
            auc_score = auc(recall, precision)
    
            result_dict[clf_name + "roc_" + feature_name].append(roc_score)
            result_dict[clf_name + "precision_" + feature_name].append(precis)
            result_dict[clf_name + "auc_" + feature_name].append(auc_score)
    
    return result_dict

def evaluate_classifier(
    clf_list = [XGBClassifier(), RandomForestClassifier()],
    clf_name_list = ["Extreme_Grandient_Boost_", "Random_Forest_"],
    feature_name_list = [
        "origin", 
        "anomaly_score", "anomaly_score_random", "anomaly_score_accurate", "anomaly_score_balance",
        "semi_supervised", "semi_supervised_random", "semi_supervised_accurate", "semi_supervised_balance"
    ]
):
    
    for evaluation in ["roc_", "precision_", "auc_"]:
        print()
        for clf_name in clf_name_list:
            for feature in feature_name_list:
                print(
                    np.round(np.mean(result_dict[clf_name + evaluation + feature]), decimals = 4),
                    evaluation + clf_name + feature
                )

In [24]:
cost_sample = pd.read_csv("cost_sample.csv")

cost_sample = cost_sample.loc[
    (~ cost_sample.country.isna()) &
    (cost_sample.cost >= 0) &
    (~ cost_sample.costType.isna()) &
    (~ cost_sample.region.isna())  &
    (~ cost_sample.usageAmount.isna()) &
    (cost_sample.usageAmount >= 0)
]

cost_sample_cat_onehot = pd.get_dummies(
    cost_sample[["description", "id", "country", "region"]],
    columns = ["description", "id", "country", "region"],
    prefix = ["description", "id", "country", "region"]
)

cost_transformed = pd.concat(
    [cost_sample_cat_onehot, cost_sample[["runningTime", "conversionRate", "usageAmount", "cost"]]],
    axis = 1
)

In [25]:
cost_transformed.head()

,description_Cloud Composer,description_Cloud Dataflow,description_Cloud Functions,description_Cloud Memorystore for Redis,description_Cloud Pub/Sub,description_Cloud SQL,description_Compute Engine,description_Stackdriver Monitoring,id_gamesys-eu-dev-sbtech-data,id_gamesys-eu-live-sbtech-data,...,region_europe-west3,region_europe-west4,region_europe-west5,region_southamerica-east1,region_us-central1,region_us-west1,runningTime,conversionRate,usageAmount,cost
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,2,0.77,2456.00,0.00
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0.79,1200.00,0.02
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,60,0.77,13326.00,0.00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,60,0.76,4446.00,0.00
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0.81,1200.00,0.02


In [26]:
pd.options.display.float_format = "{:.2f}".format
cost_transformed.describe()

,description_Cloud Composer,description_Cloud Dataflow,description_Cloud Functions,description_Cloud Memorystore for Redis,description_Cloud Pub/Sub,description_Cloud SQL,description_Compute Engine,description_Stackdriver Monitoring,id_gamesys-eu-dev-sbtech-data,id_gamesys-eu-live-sbtech-data,...,region_europe-west3,region_europe-west4,region_europe-west5,region_southamerica-east1,region_us-central1,region_us-west1,runningTime,conversionRate,usageAmount,cost
count,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,...,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00,1234.00
mean,0.05,0.00,0.00,0.00,0.03,0.01,0.88,0.03,0.02,0.04,...,0.02,0.24,0.00,0.03,0.06,0.00,77.39,0.78,198485622443355.84,0.04
std,0.22,0.05,0.07,0.03,0.16,0.10,0.32,0.16,0.14,0.19,...,0.15,0.43,0.06,0.17,0.23,0.05,100.36,0.02,2680299170069885.00,0.19
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.76,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,60.00,0.77,3599.97,0.00
50%,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,60.00,0.77,55512.42,0.00
75%,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,60.00,0.79,80852513.00,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1440.00,0.82,62493342893604864.00,3.32


In [31]:
clf_list, result_dict = initialize_classifier()

cost_transformed["label"] = 0
cost_transformed.label.loc[cost_transformed.cost >= 0.5] = 1
X_origin = cost_transformed.drop(["cost", "label"], axis = 1).values
y_origin = cost_transformed.label.values
iteration, test_size, outlier_percent = set_parameters(y_origin)

k_list_pre = [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100]
nu_list = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
n_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250]

for t in range(iteration):
    
    print("Processing trial", t + 1, "out of", iteration, "\n")
    
    # Split X and y for training and testing
    X, X_test, y, y_test = train_test_split(X_origin, y_origin, test_size = test_size)
    
    # Normalize data
    scaler = Normalizer().fit(X)
    X_norm = scaler.transform(X)
    X_test_norm = scaler.transform(X_test)
    
    feature_list_knn, train_knn, test_knn, roc_knn, precision_knn = get_results_from_KNN(
        k_list_pre, X_norm, X_test_norm, y_test, outlier_percent, "largest"
    )
    print("\n")
    feature_list_knn_mean, train_knn_mean, test_knn_mean, roc_knn_mean, precision_knn_mean = get_results_from_KNN(
        k_list_pre, X_norm, X_test_norm, y_test, outlier_percent, "mean"
    )
    print("\n")
    feature_list_knn_median, train_knn_median, test_knn_median, roc_knn_median, precision_knn_median = get_results_from_KNN(
        k_list_pre, X_norm, X_test_norm, y_test, outlier_percent, "median"
    )
    print("\n")
    feature_list_lof, train_lof, test_lof, roc_lof, precision_lof = get_results_from_LOF(
        k_list_pre, X_norm, X_test_norm, y_test, outlier_percent
    )
    print("\n")
    feature_list_svm, train_svm, test_svm, roc_svm, precision_svm = get_results_from_SVM(
        nu_list, X_norm, X_test_norm, y_test, outlier_percent
    )
    print("\n")
    feature_list_if, train_if, test_if, roc_if, precision_if = get_results_from_IF(
        n_list, X_norm, X_test_norm, y_test, outlier_percent
    )
    print("\n")
    roc = [roc_knn, roc_knn_mean, roc_knn_median, roc_lof, roc_svm, roc_if]
    precision = [precision_knn, precision_knn_mean, precision_knn_median, precision_lof, precision_svm, precision_if]
    score = [train_knn, train_knn_mean, train_knn_median, train_lof, train_svm, train_if]
    score_test = [test_knn, test_knn_mean, test_knn_median, test_lof, test_svm, test_if]
    X_train_new, X_test_new, X_train_all, X_test_all, roc_list, precision_list = generate_new_feature(
        X_norm, X_test_norm, score, score_test, roc, precision
    )

    X_train_new_rnd, X_train_all_rnd = random_selection(X_norm, X_train_new, roc_list, p = 10)
    X_test_new_rnd, X_test_all_rnd = random_selection(X_test_norm, X_test_new, roc_list, p = 10)
    X_train_new_acc, X_train_all_acc = accurate_selection(X_norm, X_train_new, roc_list, p = 10, n = 10)
    X_test_new_acc, X_test_all_acc = accurate_selection(X_test_norm, X_test_new, roc_list, p = 10, n = 10)
    X_train_new_bal, X_train_all_bal = balance_selection(X_norm, X_train_new, roc_list, p = 10)
    X_test_new_bal, X_test_all_bal = balance_selection(X_test_norm, X_test_new, roc_list, p = 10)
    
    feature_list = [
        X_norm, 
        X_train_new, X_train_new_rnd, X_train_new_acc, X_train_new_bal,
        X_train_all, X_train_all_rnd, X_train_all_acc, X_train_all_bal
    ]
    feature_test_list = [
        X_test_norm,
        X_test_new, X_test_new_rnd, X_test_new_acc, X_test_new_bal,
        X_test_all, X_test_all_rnd, X_test_all_acc, X_test_all_bal
    ]
    result_dict = train_classifier( feature_list, feature_test_list, y, y_test)
    print("\n")

/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Processing trial 1 out of 10 

largest KNN Roc and Precision in 1 Neighbors are 0.3287 and 0.0
largest KNN Roc and Precision in 2 Neighbors are 0.3076 and 0.0
largest KNN Roc and Precision in 3 Neighbors are 0.298 and 0.0
largest KNN Roc and Precision in 4 Neighbors are 0.2996 and 0.0
largest KNN Roc and Precision in 5 Neighbors are 0.295 and 0.0
largest KNN Roc and Precision in 10 Neighbors are 0.306 and 0.0
largest KNN Roc and Precision in 15 Neighbors are 0.3165 and 0.0
largest KNN Roc and Precision in 20 Neighbors are 0.3094 and 0.0
largest KNN Roc and Precision in 30 Neighbors are 0.3082 and 0.0
largest KNN Roc and Precision in 40 Neighbors are 0.3063 and 0.0
largest KNN Roc and Precision in 50 Neighbors are 0.3076 and 0.0
largest KNN Roc and Precision in 60 Neighbors are 0.3088 and 0.0
largest KNN Roc and Precision in 70 Neighbors are 0.3189 and 0.0
largest KNN Roc and Precision in 80 Neighbors are 0.3247 and 0.0
largest KNN Roc and Precision in 90 Neighbors are 0.3266 and 0.0
la

/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Local Outlier Factor Roc and Precision in 60 Neighbors are 0.5935 and 0.0
Local Outlier Factor Roc and Precision in 70 Neighbors are 0.6304 and 0.0
Local Outlier Factor Roc and Precision in 80 Neighbors are 0.6329 and 0.0
Local Outlier Factor Roc and Precision in 90 Neighbors are 0.632 and 0.0
Local Outlier Factor Roc and Precision in 100 Neighbors are 0.6163 and 0.0


Support Vector Machine Roc and Precision in 0.01 are 0.3002 and 0.0
Support Vector Machine Roc and Precision in 0.1 are 0.3425 and 0.0
Support Vector Machine Roc and Precision in 0.2 are 0.3788 and 0.0
Support Vector Machine Roc and Precision in 0.3 are 0.3953 and 0.0
Support Vector Machine Roc and Precision in 0.4 are 0.3987 and 0.0
Support Vector Machine Roc and Precision in 0.5 are 0.4036 and 0.0
Support Vector Machine Roc and Precision in 0.6 are 0.4064 and 0.0
Support Vector Machine Roc and Precision in 0.7 are 0.4108 and 0.0
Support Vector Machine Roc and Precision in 0.8 are 0.4176 and 0.0
Support Vector Machine R

/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

...

/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Isolation Forest Roc and Precision in 250 Estimators are 0.6364 and 0.0


Processing Extreme_Grandient_Boost_


/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Processing Random_Forest_


/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziye.zhou/Documents/Work/cost_anomaly_detection/tensorflow-venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [32]:
evaluate_classifier()


0.8354 roc_Extreme_Grandient_Boost_origin
0.8491 roc_Extreme_Grandient_Boost_anomaly_score
0.5431 roc_Extreme_Grandient_Boost_anomaly_score_random
0.5133 roc_Extreme_Grandient_Boost_anomaly_score_accurate
0.51 roc_Extreme_Grandient_Boost_anomaly_score_balance
0.911 roc_Extreme_Grandient_Boost_semi_supervised
0.9135 roc_Extreme_Grandient_Boost_semi_supervised_random
0.895 roc_Extreme_Grandient_Boost_semi_supervised_accurate
0.9176 roc_Extreme_Grandient_Boost_semi_supervised_balance
0.8678 roc_Random_Forest_origin
0.7903 roc_Random_Forest_anomaly_score
0.6128 roc_Random_Forest_anomaly_score_random
0.5135 roc_Random_Forest_anomaly_score_accurate
0.4951 roc_Random_Forest_anomaly_score_balance
0.8448 roc_Random_Forest_semi_supervised
0.8111 roc_Random_Forest_semi_supervised_random
0.8412 roc_Random_Forest_semi_supervised_accurate
0.8002 roc_Random_Forest_semi_supervised_balance

0.3508 precision_Extreme_Grandient_Boost_origin
0.2606 precision_Extreme_Grandient_Boost_anomaly_score
0.0167 pr